In [1]:
from sepal_ui import aoi
from component.tile import *
from component.model import *

Styles()

ResizeTrigger()

In [2]:
# Define area of interest tile
aoi_tile = aoi.AoiTile()
aoi_tile

AoiTile(align_center=True, children=[Card(children=[Html(children=['Select AOI'], tag='h2'), Flex(children=[La…

In [3]:
model = MgciModel()

In [4]:
mountain_tile = MountainTile(aoi=aoi_tile, model=model)

In [5]:
mountain_tile

MountainTile(children=[CardTitle(children=['Mountain descriptor layer']), CardText(children=[Markdown(align_ce…

In [6]:
vegetation_tile = VegetationTile(aoi_model=aoi_tile.view.model)

In [7]:
vegetation_tile

VegetationTile(children=[CardTitle(children=['Vegetation layer']), CardText(children=[Markdown(align_center=Tr…

In [ ]:
import geemap

In [ ]:
mountain_layer = mountain_tile.model.kapos_image
vegetation_layer = vegetation_tile.reclassify_tile.model.reclass_ee
asset_type = vegetation_tile.reclassify_tile.model.asset_type
aoi_geometry = aoi_tile.view.model.feature_collection.geometry()

In [ ]:
if asset_type == 'IMAGE':
    # Transform to feature collection
     mountain_layer = mountain_layer.reduceToVectors(
        reducer=ee.Reducer.countEvery(), 
        geometry=aoi_geometry, 
        scale=30,
        maxPixels=1e10
    )

In [ ]:
vegetation_layer

In [ ]:
mountain_layer

In [ ]:
import geemap

In [ ]:
from component.parameter.directory import *
from pathlib import Path

In [ ]:
# Mountain layer has to be transformed into vector


# Vegetation layer has to be an Image

In [ ]:
out_path = RESULTS_DIR/'zonal_statistics'
out_path.mkdir(parents=True, exist_ok=True)
out_file = out_path/'zonal_{}.csv'.format(aoi_tile.view.model.name)

In [ ]:
out_file

In [ ]:
geemap.zonal_statistics(
    vegetation_layer, 
    mountain_layer, 
    out_file, 
    statistics='SUM',
    bestEffort=True,
    tile_scale=4,
)

In [ ]:
vegetation_layer.reduceRegions(
    collection=mountain_layer,
    reducer=ee.Reducer.sum(),
    tileScale=16,
).getInfo()

In [ ]:
# Count the number of 

In [ ]:
import ee
ee.Initialize()

In [ ]:
ee.data.getAssetRoots()[0]['id']

In [ ]:
params = {
    'image': mountain_layer,
    'assetId': 'projects/earthengine-legacy/assets/users/dafguerrerom/COL_ANT_Kapos2',
    'description': 'Col_ANT_Kapos2',
    'scale': 30,
    'maxPixels': 1e13,
}

task = ee.batch.Export.image.toAsset(**params)
task.start()